# Import of Data

In order to draw several comparisons of each genre and month of release between reviews, gross revenue, I first imported and joined 3 different datasets, two from the popular rating site IMDband the other from The Numbers. This will allow a concrete picture of what genre of movie is successful when it comes to these variables, as well as view trends in time using the release date information.

In [1]:
import pandas as pd
import numpy as np

In [2]:
revenue_df = pd.read_csv('data/tn.movie_budgets.csv.gz')
revenue_df.head()

id  release_date                                        movie  \
0   1  Dec 18, 2009                                       Avatar   
1   2  May 20, 2011  Pirates of the Caribbean: On Stranger Tides   
2   3   Jun 7, 2019                                 Dark Phoenix   
3   4   May 1, 2015                      Avengers: Age of Ultron   
4   5  Dec 15, 2017            Star Wars Ep. VIII: The Last Jedi   

  production_budget domestic_gross worldwide_gross  
0      $425,000,000   $760,507,625  $2,776,345,279  
1      $410,600,000   $241,063,875  $1,045,663,875  
2      $350,000,000    $42,762,350    $149,762,350  
3      $330,600,000   $459,005,868  $1,403,013,963  
4      $317,000,000   $620,181,382  $1,316,721,747

In [3]:
revenue_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [4]:
genre_df = pd.read_csv('data/imdb.title.basics.csv.gz')
genre_df.head()

tconst                    primary_title              original_title  \
0  tt0063540                        Sunghursh                   Sunghursh   
1  tt0066787  One Day Before the Rainy Season             Ashad Ka Ek Din   
2  tt0069049       The Other Side of the Wind  The Other Side of the Wind   
3  tt0069204                  Sabse Bada Sukh             Sabse Bada Sukh   
4  tt0100275         The Wandering Soap Opera       La Telenovela Errante   

   start_year  runtime_minutes                genres  
0        2013            175.0    Action,Crime,Drama  
1        2019            114.0       Biography,Drama  
2        2018            122.0                 Drama  
3        2018              NaN          Comedy,Drama  
4        2017             80.0  Comedy,Drama,Fantasy

In [5]:
genre_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [6]:
rating_df = pd.read_csv('data/imdb.title.ratings.csv.gz')
rating_df.head()

tconst  averagerating  numvotes
0  tt10356526            8.3        31
1  tt10384606            8.9       559
2   tt1042974            6.4        20
3   tt1043726            4.2     50352
4   tt1060240            6.5        21

In [7]:
rating_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


# Merge

In order to have all the values we need for each specific movie, we first have to merge the data frames from the same iMDB source by the in common `tconst` id. I am using an inner join to exlude any unmatched data and ensure there are no missing values for the data features.

In [8]:
df1 = pd.merge(rating_df, genre_df, on=["tconst"], how ="inner")
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73856 entries, 0 to 73855
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tconst           73856 non-null  object 
 1   averagerating    73856 non-null  float64
 2   numvotes         73856 non-null  int64  
 3   primary_title    73856 non-null  object 
 4   original_title   73856 non-null  object 
 5   start_year       73856 non-null  int64  
 6   runtime_minutes  66236 non-null  float64
 7   genres           73052 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 5.1+ MB


Then we need to join this dataset with the `revenue_df` by common values found in the columns `movie` and `primary_title` 

In [9]:
final_df = pd.merge(revenue_df, df1, left_on="movie", right_on="primary_title", how ="inner")
final_df.head(3)

id  release_date                                        movie  \
0   1  Dec 18, 2009                                       Avatar   
1   2  May 20, 2011  Pirates of the Caribbean: On Stranger Tides   
2   3   Jun 7, 2019                                 Dark Phoenix   

  production_budget domestic_gross worldwide_gross     tconst  averagerating  \
0      $425,000,000   $760,507,625  $2,776,345,279  tt1775309            6.1   
1      $410,600,000   $241,063,875  $1,045,663,875  tt1298650            6.6   
2      $350,000,000    $42,762,350    $149,762,350  tt6565702            6.0   

   numvotes                                primary_title  \
0        43                                       Avatar   
1    447624  Pirates of the Caribbean: On Stranger Tides   
2     24451                                 Dark Phoenix   

                                original_title  start_year  runtime_minutes  \
0                                        Abatâ        2011             93.0   
1  Pirates of the Caribbean: On Stranger Tides        2011            136.0   
2                                 Dark Phoenix        2019            113.0   

                     genres  
0                    Horror  
1  Action,Adventure,Fantasy  
2   Action,Adventure,Sci-Fi

In [10]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2875 entries, 0 to 2874
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 2875 non-null   int64  
 1   release_date       2875 non-null   object 
 2   movie              2875 non-null   object 
 3   production_budget  2875 non-null   object 
 4   domestic_gross     2875 non-null   object 
 5   worldwide_gross    2875 non-null   object 
 6   tconst             2875 non-null   object 
 7   averagerating      2875 non-null   float64
 8   numvotes           2875 non-null   int64  
 9   primary_title      2875 non-null   object 
 10  original_title     2875 non-null   object 
 11  start_year         2875 non-null   int64  
 12  runtime_minutes    2757 non-null   float64
 13  genres             2867 non-null   object 
dtypes: float64(2), int64(3), object(9)
memory usage: 336.9+ KB


As seen above, there are just over 2,800 unique movies in the remaining data set.

# Cleaning Columns & Updating Type

Next, in order to use the values in projects analysis, I update the key columns to their correct variable type such as int and datetime.

In [11]:
final_df["release_date"] = pd.to_datetime(final_df["release_date"])

In [12]:
final_df["averagerating"] = final_df["averagerating"].astype(float)

For the currency conversions we have to remove the ',' and '$' string values to properly convert the columns to int. To do this I use the below function rather than  a lambda function as we have several columns we need to do this on. Currency function provided by [Practical Business Python](https://pbpython.com/pandas_dtypes.html)

In [13]:
def convert_currency(val):
    new_val = val.replace(',','').replace('$', '')
    return int(new_val)

In [14]:
final_df["production_budget"] = final_df["production_budget"].apply(convert_currency)
final_df["domestic_gross"] = final_df["domestic_gross"].apply(convert_currency)
final_df["worldwide_gross"] = final_df["worldwide_gross"].apply(convert_currency)

In [15]:
final_df.head(1)

id release_date   movie  production_budget  domestic_gross  \
0   1   2009-12-18  Avatar          425000000       760507625   

   worldwide_gross     tconst  averagerating  numvotes primary_title  \
0       2776345279  tt1775309            6.1        43        Avatar   

  original_title  start_year  runtime_minutes  genres  
0          Abatâ        2011             93.0  Horror

# Feature Engineering

Next I make a feature column to determe the actual revenue of each movie by subtracting the `worldwide_gross` from the `production_budget`. I use the worldwide box office rather than the domestic as it gives a better overview of how much the movie made overall. This will assist in the analysis of which genres are more profitable.

In [16]:
final_df["gross_revenue"] = final_df["worldwide_gross"] - final_df["production_budget"]

Each movie can be attributed at maximum three different genre categories. To enable analysis into the different genres, I split and explode these values into their own rows.

In [17]:
final_df["genres"] = final_df["genres"].str.split(pat = ',')

In [18]:
final_df = final_df.explode("genres")

I make the data easier to comprehend by removing columns that are unnessecary for future analysis.

In [19]:
final_df = final_df[["id", "release_date", "movie", "averagerating", "genres", "gross_revenue"]]

In [20]:
final_df = final_df.set_index("id")
final_df.head()

release_date                                        movie  averagerating  \
id                                                                            
1    2009-12-18                                       Avatar            6.1   
2    2011-05-20  Pirates of the Caribbean: On Stranger Tides            6.6   
2    2011-05-20  Pirates of the Caribbean: On Stranger Tides            6.6   
2    2011-05-20  Pirates of the Caribbean: On Stranger Tides            6.6   
3    2019-06-07                                 Dark Phoenix            6.0   

       genres  gross_revenue  
id                            
1      Horror     2351345279  
2      Action      635063875  
2   Adventure      635063875  
2     Fantasy      635063875  
3      Action     -200237650

In [21]:
final_df.to_csv('data/final_clean.csv')